In [21]:
!pip install dash
import requests
import pandas as pd
import plotly.express as px
from bs4 import BeautifulSoup
import dash
from dash import dcc, html, Input, Output

# Map month codes (M01, M02, etc.) to actual month names
month_mapping = {
    "M01": "January", "M02": "February", "M03": "March", "M04": "April", "M05": "May", "M06": "June",
    "M07": "July", "M08": "August", "M09": "September", "M10": "October", "M11": "November", "M12": "December"
}

# Function to fetch and parse the data from BLS table
def fetch_bls_table_data():
    # URL of the BLS table
    url = "https://data.bls.gov/dataViewer/view/timeseries/LNS11000000"

    # Make a GET request to fetch the page content
    response = requests.get(url)

    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the table with the id "seriesDataTable1"
    table = soup.find("table", {"id": "seriesDataTable1"})

    # Check if the table was found
    if table:
        rows = table.find_all("tr")
        data = []

        # Loop through each row in the table and extract relevant data
        for row in rows:
            cols = row.find_all("td")
            if len(cols) > 0:
                # Extract year and period (Month Code)
                year = cols[0].text.strip()
                period = cols[1].text.strip()  # e.g., M01, M02, etc.
                value = float(cols[3].text.strip().replace(",","").split("\r")[0])

                # Convert the month period to YYYY-MM format
                try:
                    # Get the correct month code (e.g., M01 -> 01, M02 -> 02, etc.)
                    month_code = period[1:]  # Get the numeric part after 'M'
                    concatenated_date = f"{month_code}-{year}"  # Create a date in MM-YYYY format

                    # Append the year, month, and value to the data list
                    data.append({
                        "date": concatenated_date,
                        "value": float(value)
                    })
                except ValueError:
                    # Handle cases where the date format is unexpected
                    print(f"Skipping row with invalid date format: {period} {year}")
                    continue  # Skip this row and continue with the next

        # Convert the list of data into a DataFrame
        df = pd.DataFrame(data)
        df["date"] = pd.to_datetime(df["date"], format="%m-%Y")  # Convert to datetime format
        return df
    else:
        print("Table not found.")
        return pd.DataFrame()

# Calculate percentage change for MoM and YoY
def calculate_percentage_change(df, comparison_type):
    if comparison_type == "MoM":
        df["change"] = df["value"].pct_change() * 100  # Month over Month change
        title = "Month Over Month % Change"
    elif comparison_type == "YoY":
        df["change"] = df["value"].pct_change(periods=12) * 100  # Year over Year change
        title = "Year Over Year % Change"

    return df, title

# Initialize Dash app
app = dash.Dash(__name__)

# Fetch the data
df = fetch_bls_table_data()

# Check if the DataFrame is not empty before plotting
if not df.empty:
    # Initial graph using 'MoM' as default
    df, title = calculate_percentage_change(df, "MoM")

    app.layout = html.Div([
        html.H1("BLS Employment Data"),

        # Dropdown for comparison type
        html.Label("Select Comparison Type:"),
        dcc.Dropdown(
            id="comparison-type",
            options=[
                {"label": "Month Over Month", "value": "MoM"},
                {"label": "Year Over Year", "value": "YoY"}
            ],
            value="MoM",  # Default value
        ),

        # Graph for the employment data
        dcc.Graph(id="jobs-graph", figure=px.line(df, x="date", y="change", title=title,
                                                  labels={"date": "Date", "change": "% Change in Employment Rate"},
                                                  template="plotly_dark").update_layout(
            yaxis=dict(title="% Change in Employment Rate (Civilian)"))
        ),
    ])

    # Callback to update graph based on selected comparison type
    @app.callback(
        Output("jobs-graph", "figure"),
        Input("comparison-type", "value"),
    )
    def update_graph(comparison_type):
        df_copy = df.copy()  # Ensure we don't modify original data
        df_copy, title = calculate_percentage_change(df_copy, comparison_type)

        # Create the graph
        fig = px.line(df_copy, x="date", y="change", title=title,
                      labels={"date": "Date", "change": "% Change in Employment Rate"},
                      template="plotly_dark")

        # Update layout of the graph
        fig.update_layout(
            yaxis=dict(title="% Change in Employment Rate"),
        )

        return fig

    # Run the app
    if __name__ == "__main__":
        app.run_server(debug=True)

else:
    print("No data available for the graph.")


<IPython.core.display.Javascript object>